# Load from Features Notebook

In [1]:
import dill
dill.load_session("Notebook_Saves/features.db")

In [2]:
import lightgbm as lgb

# Model

In [3]:
X = training.drop(columns=['user_id_hash', 'user_purchase_binary_7_days',
                           'user_purchase_binary_14_days', 'country', 'os_name'])
y = training['user_purchase_binary_7_days']
X_test = submit.drop(columns=['user_id_hash', 'user_purchase_binary_7_days',
                              'user_purchase_binary_14_days', 'country', 'os_name'])

In [4]:
params = {'num_leaves': 8,
          'min_data_in_leaf': 40,
          'objective': 'binary',
          'max_depth': 16,
          'learning_rate': 0.01,
          'boosting': 'gbdt',
          'bagging_freq': 5,
          'bagging_fraction': 0.8,
          'feature_fraction': 0.8,
          'bagging_seed': 11,
          'reg_alpha': 1.7,
          'reg_lambda': 5,
          'random_state': 42,
          'metric': 'auc',
          'verbosity': -1,
          'subsample': 0.8,
#          'min_gain_to_split': 0.01,
#          'min_child_weight': 20,
          'num_threads': 4}

In [5]:
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)

In [6]:
seven_day_pred = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)

    model = lgb.train(params, train_data, num_boost_round=20000,
                      valid_sets=[train_data, valid_data],
                      verbose_eval=300,
                      early_stopping_rounds=200)

    #y_pred_valid = model.predict(X_valid)
    seven_day_pred += model.predict(X_test,
                                    num_iteration=model.best_iteration)/n_fold

Fold 0 started at Wed Feb 20 20:14:24 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.99427	valid_1's auc: 0.993424
[600]	training's auc: 0.994539	valid_1's auc: 0.993691
[900]	training's auc: 0.994703	valid_1's auc: 0.993807
[1200]	training's auc: 0.994839	valid_1's auc: 0.993852
[1500]	training's auc: 0.994948	valid_1's auc: 0.993882
[1800]	training's auc: 0.995053	valid_1's auc: 0.99389
[2100]	training's auc: 0.995146	valid_1's auc: 0.993896
[2400]	training's auc: 0.995236	valid_1's auc: 0.993895
Early stopping, best iteration is:
[2203]	training's auc: 0.995176	valid_1's auc: 0.993898
Fold 1 started at Wed Feb 20 20:16:07 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.994205	valid_1's auc: 0.993751
[600]	training's auc: 0.99448	valid_1's auc: 0.993988
[900]	training's auc: 0.994638	valid_1's auc: 0.994077
[1200]	training's auc: 0.994769	valid_1's auc: 0.994117
[1500]	training's auc: 0.994881	va

In [7]:
X = training.drop(columns=['user_id_hash', 'user_purchase_binary_7_days',
                           'user_purchase_binary_14_days', 'country', 'os_name'])
y = training['user_purchase_binary_14_days']
X_test = submit.drop(columns=['user_id_hash', 'user_purchase_binary_7_days',
                              'user_purchase_binary_14_days', 'country', 'os_name'])

In [8]:
fourteen_day_pred = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)

    model = lgb.train(params, train_data, num_boost_round=20000,
                      valid_sets=[train_data, valid_data],
                      verbose_eval=300,
                      early_stopping_rounds=200)

    #y_pred_valid = model.predict(X_valid)
    fourteen_day_pred += model.predict(X_test,
                                       num_iteration=model.best_iteration)/n_fold

Fold 0 started at Wed Feb 20 20:21:40 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.983424	valid_1's auc: 0.981831
[600]	training's auc: 0.984556	valid_1's auc: 0.983256
[900]	training's auc: 0.985182	valid_1's auc: 0.983808
[1200]	training's auc: 0.985573	valid_1's auc: 0.984025
[1500]	training's auc: 0.985902	valid_1's auc: 0.984162
[1800]	training's auc: 0.986177	valid_1's auc: 0.984205
[2100]	training's auc: 0.986468	valid_1's auc: 0.984274
Early stopping, best iteration is:
[2050]	training's auc: 0.986427	valid_1's auc: 0.984286
Fold 1 started at Wed Feb 20 20:23:24 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.983665	valid_1's auc: 0.980812
[600]	training's auc: 0.984814	valid_1's auc: 0.982053
[900]	training's auc: 0.985487	valid_1's auc: 0.982411
[1200]	training's auc: 0.985887	valid_1's auc: 0.982657
[1500]	training's auc: 0.986162	valid_1's auc: 0.982739
[1800]	training's auc: 0.986468

In [9]:
submission['user_purchase_binary_7_days'] = seven_day_pred
submission['user_purchase_binary_14_days'] = fourteen_day_pred

In [10]:
submission.to_csv('submission0220_2.csv', index=False)

In [129]:
!kaggle competitions submit predict-in-app-purchase -f submission0217.csv -m "Using full 11 features and model on lgbm"

100%|██████████████████████████████████████| 32.6M/32.6M [00:02<00:00, 16.1MB/s]
Successfully submitted to Predict in-app purchases